In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [1]:
metric_info = [
    ("container_cpu_usage_seconds_total", "cpu"),
    ("container_memory_usage_bytes", "mem")
]
pod_info = [
    ("adservice-b64d6db99-6qlr7", "adservice"),
    ("cartservice-7bdffbf7-nzwgf", "cartservice"),
    ("checkoutservice-7988bbf57d-ql2vx", "checkoutservice"),
    ("currencyservice-6f58f94d86-mf5pn", "currencyservice"),
    ("emailservice-965c88745-2zv7z", "emailservice"),
    ("frontend-7cc759d45-7l2ff", "frontend"),
    ("paymentservice-6cff547576-h4qsq", "paymentservice"),
    ("productcatalogservice-6b7556db5-t2sfh", "productcatalogservice"),
    ("recommendationservice-85555c666b-hhp27", "recommendationservice"),
    ("redis-cart-5477c6b974-vsvd5", "redis-cart"),
    ("shippingservice-68fc55fb6d-plfqx", "shippingservice"),
]

In [21]:
window = 12
horizont = 1

In [22]:
node_df = pd.read_csv("../data/pod_metrics.csv", index_col="timestamp")
node_df

,adservice-cpu,cartservice-cpu,checkoutservice-cpu,currencyservice-cpu,emailservice-cpu,frontend-cpu,paymentservice-cpu,productcatalogservice-cpu,recommendationservice-cpu,redis-cart-cpu,...,cartservice-mem,checkoutservice-mem,currencyservice-mem,emailservice-mem,frontend-mem,paymentservice-mem,productcatalogservice-mem,recommendationservice-mem,redis-cart-mem,shippingservice-mem
timestamp,,,,,,,,,,,,,,,,,,,,,
1708449998,2.837169,2.837169,2.837169,2.837169,2.837169,2.837169,2.837169,2.837169,2.837169,2.837169,...,6.457040e+08,6.457040e+08,6.457040e+08,6.457040e+08,6.457040e+08,6.457040e+08,6.457040e+08,6.457040e+08,6.457040e+08,6.457040e+08
1708450898,2.684841,2.684841,2.684841,2.684841,2.684841,2.684841,2.684841,2.684841,2.684841,2.684841,...,6.785983e+08,6.785983e+08,6.785983e+08,6.785983e+08,6.785983e+08,6.785983e+08,6.785983e+08,6.785983e+08,6.785983e+08,6.785983e+08
1708451798,2.697294,2.697294,2.697294,2.697294,2.697294,2.697294,2.697294,2.697294,2.697294,2.697294,...,6.882176e+08,6.882176e+08,6.882176e+08,6.882176e+08,6.882176e+08,6.882176e+08,6.882176e+08,6.882176e+08,6.882176e+08,6.882176e+08
1708452698,2.688250,2.688250,2.688250,2.688250,2.688250,2.688250,2.688250,2.688250,2.688250,2.688250,...,7.062813e+08,7.062813e+08,7.062813e+08,7.062813e+08,7.062813e+08,7.062813e+08,7.062813e+08,7.062813e+08,7.062813e+08,7.062813e+08
1708453598,2.689835,2.689835,2.689835,2.689835,2.689835,2.689835,2.689835,2.689835,2.689835,2.689835,...,7.159098e+08,7.159098e+08,7.159098e+08,7.159098e+08,7.159098e+08,7.159098e+08,7.159098e+08,7.159098e+08,7.159098e+08,7.159098e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1709051198,3.345288,3.345288,3.345288,3.345288,3.345288,3.345288,3.345288,3.345288,3.345288,3.345288,...,8.273749e+08,8.273749e+08,8.273749e+08,8.273749e+08,8.273749e+08,8.273749e+08,8.273749e+08,8.273749e+08,8.273749e+08,8.273749e+08
1709052098,3.360023,3.360023,3.360023,3.360023,3.360023,3.360023,3.360023,3.360023,3.360023,3.360023,...,8.221986e+08,8.221986e+08,8.221986e+08,8.221986e+08,8.221986e+08,8.221986e+08,8.221986e+08,8.221986e+08,8.221986e+08,8.221986e+08
1709052998,3.371598,3.371598,3.371598,3.371598,3.371598,3.371598,3.371598,3.371598,3.371598,3.371598,...,8.146160e+08,8.146160e+08,8.146160e+08,8.146160e+08,8.146160e+08,8.146160e+08,8.146160e+08,8.146160e+08,8.146160e+08,8.146160e+08


In [27]:
X_node = []
y_node = []

for i in tqdm(range(window, len(node_df) - horizont + 1)):
    node_window = node_df.iloc[i - window: i]
    node_horizont = node_df.iloc[i + horizont - 1]

    x, y = [], []
    for (_, pod_name) in pod_info:
        x.append(node_window[[f"{pod_name}-cpu", f"{pod_name}-mem"]].to_numpy())
        y.append(node_horizont[[f"{pod_name}-cpu", f"{pod_name}-mem"]].to_numpy())
    X_node.append(np.array(x).swapaxes(0, 1))
    y_node.append(np.array(y))
X_node = np.array(X_node)
y_node = np.array(y_node)
X_node[:, :, :, 1] = X_node[:, :, :, 1] / 1e6
y_node[:, :, 1] = y_node[:, :, 1] / 1e6
# np.savez(os.path.join(args.output, "node_features.npz"), X=X_node, y=y_node)

100%|██████████| 661/661 [00:02<00:00, 272.95it/s]


In [28]:
X_node.shape

(661, 12, 11, 2)